In [1]:
# allocate_fleet_by_origin.py
# 依赖：pandas  (pip install pandas)

from pathlib import Path
import pandas as pd
import numpy as np

# ================== 配置（按需修改） ==================
OD_CSV            = r"../data/od_matrix.csv"         # 输入 OD 表（CSV/TSV；列含 t,i,j,demand）
ZONES_MASK_CSV    = r""                               # 可选：只在这些 zones 内分配（CSV 至少含列 zone）；留空表示不用mask
OUT_CSV           = r"../data/fleet_init.csv"   # 输出：zone,soc,count

FLEET_SIZE        = 200                              # 你的车队规模（整数）
FILTER_T_VALUES   = None                              # 例如 {1,2,3} 仅按这些时段汇总；None 表示所有 t
DROP_ZERO_DEMAND  = True                              # 是否剔除出发地总需求为 0 的区（一般 True）

SOC_MODE          = "fixed"                           # "fixed" 固定一个SOC；也可拓展其他模式
SOC_FIXED_VALUE   = 80                                # 常见研究默认：80% 初始电量（0~100）
# =====================================================

def read_od(path: str) -> pd.DataFrame:
    # 兼容逗号/制表符
    df = pd.read_csv(path, sep=None, engine="python",
                     dtype={"i": str, "j": str})
    # 规范列名
    cols = {c.lower(): c for c in df.columns}
    need = ["t","i","j","demand"]
    miss = [c for c in need if c not in cols]
    if miss:
        raise ValueError(f"OD缺少必要列 {need}；实际列：{list(df.columns)}")

    od = df[[cols["t"], cols["i"], cols["j"], cols["demand"]]].copy()
    od.columns = ["t","i","j","demand"]
    # 类型
    od["t"] = pd.to_numeric(od["t"], errors="coerce").astype("Int64")
    od["demand"] = pd.to_numeric(od["demand"], errors="coerce").fillna(0.0)
    return od

def load_zones_mask(path: str) -> set[str]:
    z = pd.read_csv(path, dtype={"zone": str})
    if "zone" not in z.columns:
        raise ValueError(f"{path} 需要列 'zone'")
    return set(z["zone"].astype(str))

def hamilton_allocate(weights: pd.Series, total: int) -> pd.Series:
    """
    最大剩余法（Hamilton method）
    weights: 各区权重（非负，和>0）
    total:   需要分配的总整数
    返回：各区整数份额，索引与 weights 对齐，和为 total
    """
    # 原始小数份额
    raw = weights * total
    base = np.floor(raw).astype(int)
    remain = int(total - base.sum())

    if remain < 0:
        raise ValueError("总基数超过 fleet_size？请检查 weights 或 total。")

    # 余数降序，若并列用原始权重降序，再按索引稳定
    frac = (raw - base).rename("frac")
    sort_key = pd.DataFrame({
        "frac": frac,
        "weight": weights
    })
    order = sort_key.sort_values(["frac","weight"], ascending=[False, False]).index

    alloc = base.copy()
    if remain > 0:
        top = list(order[:remain])
        alloc.loc[top] += 1
    return alloc

def main():
    # 1) 读取 OD
    od = read_od(OD_CSV)
    if FILTER_T_VALUES:
        od = od[od["t"].isin(FILTER_T_VALUES)].copy()

    # 2) 聚合出发地需求（所有目的地累加）
    demand_by_i = (
        od.groupby("i", dropna=False)["demand"]
          .sum()
          .astype(float)
    )

    # 3) 可选：应用 zones mask（只在这些出发地里分配）
    if ZONES_MASK_CSV:
        keep = load_zones_mask(ZONES_MASK_CSV)
        demand_by_i = demand_by_i[demand_by_i.index.astype(str).isin(keep)]

    if DROP_ZERO_DEMAND:
        demand_by_i = demand_by_i[demand_by_i > 0]

    if demand_by_i.empty:
        raise RuntimeError("没有可分配的出发地（可能全部需求为0或mask过滤为空）。")

    total_demand = demand_by_i.sum()
    if total_demand <= 0:
        raise RuntimeError("总需求和为0，无法按比例分配。")

    # 4) 计算权重并按 Hamilton method 分配整数车辆数
    weights = demand_by_i / total_demand
    counts = hamilton_allocate(weights, int(FLEET_SIZE))

    # 5) 生成 soc 列（默认固定 80；可自行扩展成分布）
    if SOC_MODE == "fixed":
        soc_series = pd.Series(SOC_FIXED_VALUE, index=counts.index, dtype=int)
    else:
        raise NotImplementedError("目前仅实现 SOC_MODE='fixed'。")

    # 6) 输出 zone,soc,count
    out = pd.DataFrame({
        "zone": counts.index.astype(str),
        "soc": soc_series.astype(int),
        "count": counts.astype(int)
    }).sort_values("zone", kind="stable").reset_index(drop=True)

    # 校验和
    assert int(out["count"].sum()) == int(FLEET_SIZE), "分配后的总数不等于 fleet_size！"

    Path(OUT_CSV).parent.mkdir(parents=True, exist_ok=True)
    out.to_csv(OUT_CSV, index=False)
    print(f"Saved: {OUT_CSV}  rows={len(out)}")
    print("总车队：", FLEET_SIZE, "；分配到的出发地数量：", len(out), "；总需求：", total_demand, "；总区域数：", len(demand_by_i))
    print(out.head())

if __name__ == "__main__":
    main()


Saved: ../data/fleet_init.csv  rows=205
总车队： 200 ；分配到的出发地数量： 205 ；总需求： 119740.0 ；总区域数： 205
       zone  soc  count
0  00000308   80      0
1  00000317   80      0
2  00000319   80      1
3  00000320   80      0
4  00000321   80      1
